In [1]:
import numpy as np
import pandas as pd
import itertools
import random
from tqdm import tqdm
from ast import literal_eval
import time

from sklearn.multiclass import OneVsRestClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from torch.autograd import Variable
device = torch.device("cpu")

# import pickle
import joblib

import matplotlib.pyplot as plt

In [2]:
config_dict_1 = {
    0: "fair",
    1: "cheat-1_no-cooldown",
    2: "cheat-2_no-dash",
    3: "cheat-3_inf-slowmo",
    4: "cheat-4_huge-damage",
    5: "cheat-5_frozen-ninja",
}

config_dict_2 = {
    0: "fair_new",
    1: "cheat-1_new",
    2: "cheat-2_new",
    3: "cheat-3_new",
    4: "cheat-4_new",
    5: "cheat-5_new",
}

In [3]:
def preprocess_gt_csv(data_name):
    filename = f"train_data/gt/{data_name}.csv"

    columns_to_eval = [
        "global_mouse_position",
        "button_cooldown_times",
        "logical_frames_since_last_button_press",
        "global_position",
        "velocity",
        "fruits_cut_this_frame_list",
        "fruits_hit_ninjas_ass_this_frame_list",
        "fruits_on_screen_this_frame_list",
        "fruits_spawned_this_frame_list",
        "stupid_fucking_fruits_that_died_this_frame_list",
    ]

    df = pd.read_csv(filename, converters={column: literal_eval for column in columns_to_eval})

    df["fruits_spawned_num"] = df["fruits_spawned_this_frame_list"].str.len()
    df["fruits_on_screen_num"] = df["fruits_on_screen_this_frame_list"].str.len()
    df["fruits_hit_num"] = df["fruits_hit_ninjas_ass_this_frame_list"].str.len()
    df["fruits_missed_num"] = df["stupid_fucking_fruits_that_died_this_frame_list"].str.len()

    projectiles = ["Apple", "Banana", "Grape", "Pineapple", "Watermelon", "Coconut", "Cherry"]

    df["fruit_name_spawned"] = df["fruits_spawned_this_frame_list"].apply(lambda x: projectiles.index(x[0]["name"]) if len(x) != 0 else -1)

    for butt in ["b1", "b2", "b3", "b4"]:
        df[butt] = df["button_cooldown_times"].apply(lambda j: j[butt])

    general_needed = [
        "cheat_flag",
        "frame_count",
        "session_id",
        "score",
        "combo",
        "is_combo_going",
        "time_left_seconds",
        "can_dash",
    ]
    buttons_needed = [
        "b1", "b2", "b3", "b4",
    ]

    df = df[general_needed + buttons_needed]
    df[general_needed] = df[general_needed].astype(int)

    if data_name == "cheat-4_huge-damage":  # its too fucking huge
        df["score"] *= 0.0001

    return df

In [4]:
def preprocess_synthetic_csv(data_name):
    filename = f"train_data/synthetic/{data_name}_50-games.csv"
    df = pd.read_csv(filename).drop(["Unnamed: 0"], axis=1)
    return df

In [5]:
synthetic_data = pd.concat([preprocess_synthetic_csv(data_name) for data_name in config_dict_1.values()])

X_synthetic = synthetic_data.drop(["cheat_flag", "session_id"], axis=1)
y_synthetic = synthetic_data["cheat_flag"]

X_synthetic.shape, y_synthetic.shape

((1979694, 10), (1979694,))

In [6]:
gt_data_1 = pd.concat([preprocess_gt_csv(data_name) for data_name in config_dict_1.values()])
gt_data_2 = pd.concat([preprocess_gt_csv(data_name) for data_name in config_dict_2.values()])
gt_data = pd.concat([gt_data_1, gt_data_2])

X_gt = gt_data.drop(["cheat_flag", "session_id"], axis=1)
y_gt = gt_data["cheat_flag"]

X_gt.shape, y_gt.shape

((373128, 10), (373128,))

In [7]:
mix_num = len(X_gt) // 4

gt_idx = np.random.choice(len(X_gt), replace=False, size=mix_num)

gt_rest = [i for i in np.arange(len(X_gt)) if i not in gt_idx]

X_train_mixed = pd.concat([X_synthetic, X_gt.iloc[gt_idx]])
y_train_mixed = pd.concat([y_synthetic, y_gt.iloc[gt_idx]])

X_test = X_gt.iloc[gt_rest]
y_test = y_gt.iloc[gt_rest]

y_train_mixed = nn.functional.one_hot(torch.tensor(y_train_mixed.to_numpy()), len(config_dict_1))
y_test = nn.functional.one_hot(torch.tensor(y_test.to_numpy()), len(config_dict_1))

X_train_mixed.shape, y_train_mixed.shape, X_test.shape, y_test.shape

((2072976, 10),
 torch.Size([2072976, 6]),
 (279846, 10),
 torch.Size([279846, 6]))

In [8]:
# X_train, X_val, y_train, y_val = train_test_split(
#     X_train_mixed, y_train_mixed, test_size=0.4, random_state=42, shuffle=True
# )
# X_train.shape, X_val.shape, y_train.shape, y_val.shape

In [9]:
train_set = TensorDataset(
    torch.FloatTensor(X_train_mixed.to_numpy()).unsqueeze(1),
    y_train_mixed,
)
train_loader = DataLoader(train_set, batch_size=6_000, shuffle=True)

test_set = TensorDataset(
    torch.FloatTensor(X_test.to_numpy()).unsqueeze(1),
    y_test,
)
test_loader = DataLoader(test_set, batch_size=6_000, shuffle=True)

In [10]:
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier

names = [
    "Decision Tree",
    "Random Forest",
    "Gaussian Naive Bayes",
]

classifiers = [
    DecisionTreeClassifier(random_state=42),
    RandomForestClassifier(random_state=42),
    GaussianNB(),
]

In [11]:
for name, clf in zip(names, classifiers):
    print(f"{name} model:")
    clf = OneVsRestClassifier(clf)

    train_acc = 0
    start = time.time()

    for X, y in tqdm(train_loader):
        clf.fit(X.numpy().squeeze(), y.numpy())
        train_acc += clf.score(X.numpy().squeeze(), y.numpy())

    end = time.time()
    train_acc /= len(train_loader)

    test_acc = 0
    for X, y in tqdm(test_loader):
        test_acc += clf.score(X.numpy().squeeze(), y.numpy())
    test_acc /= len(test_loader)

    snake_case_name = "_".join(name.lower().split(" "))
    joblib.dump(clf, f"../microservices/DataManager/weights/multiclass_more/ovr/{snake_case_name}.joblib")
    print(f"Saved model.")

    print(f"Train time: {(end - start):.1f} sec,  Train score: {train_acc:.3f},  Gt score: [ {test_acc:.3f} ]")
    print("-" * 150)

Decision Tree model:


100%|██████████| 47/47 [00:01<00:00, 25.77it/s]


Saved model.
Train time: 65.9 sec,  Train score: 1.000,  Gt score: [ 0.546 ]
------------------------------------------------------------------------------------------------------------------------------------------------------
Random Forest model:


100%|██████████| 47/47 [00:10<00:00,  4.29it/s]


Saved model.
Train time: 1526.0 sec,  Train score: 1.000,  Gt score: [ 0.573 ]
------------------------------------------------------------------------------------------------------------------------------------------------------
Gaussian Naive Bayes model:


100%|██████████| 47/47 [00:01<00:00, 23.81it/s]

Saved model.
Train time: 18.9 sec,  Train score: 0.078,  Gt score: [ 0.179 ]
------------------------------------------------------------------------------------------------------------------------------------------------------


In [12]:
for name, clf in zip(names, classifiers):
    print(f"{name} model:")
    clf = OneVsOneClassifier(clf)

    train_acc = 0
    start = time.time()

    for X, y in tqdm(train_loader):
        y = torch.argmax(y, dim=1)
        clf.fit(X.numpy().squeeze(), y.numpy())
        train_acc += clf.score(X.numpy().squeeze(), y.numpy())

    end = time.time()
    train_acc /= len(train_loader)

    test_acc = 0
    for X, y in tqdm(test_loader):
        y = torch.argmax(y, dim=1)
        test_acc += clf.score(X.numpy().squeeze(), y.numpy())
    test_acc /= len(test_loader)

    snake_case_name = "_".join(name.lower().split(" "))
    joblib.dump(clf, f"../microservices/DataManager/weights/multiclass_more/ovo/{snake_case_name}.joblib")
    print(f"Saved model.")

    print(f"Train time: {(end - start):.1f} sec,  Train score: {train_acc:.3f},  Gt score: [ {test_acc:.3f} ]")
    print("-" * 150)

Decision Tree model:


100%|██████████| 47/47 [00:02<00:00, 23.03it/s]


Saved model.
Train time: 46.1 sec,  Train score: 1.000,  Gt score: [ 0.662 ]
------------------------------------------------------------------------------------------------------------------------------------------------------
Random Forest model:


100%|██████████| 47/47 [00:40<00:00,  1.15it/s]


Saved model.
Train time: 1357.8 sec,  Train score: 1.000,  Gt score: [ 0.687 ]
------------------------------------------------------------------------------------------------------------------------------------------------------
Gaussian Naive Bayes model:


100%|██████████| 47/47 [00:02<00:00, 17.38it/s]


Saved model.
Train time: 26.6 sec,  Train score: 0.495,  Gt score: [ 0.577 ]
------------------------------------------------------------------------------------------------------------------------------------------------------


In [13]:
input_size = 10
hidden_size = 50
output_size = len(config_dict_1)
num_layers = 1

learning_rate = 0.001
num_epochs = 50

In [14]:
from microservices.DataManager.models.lstm import LSTMModel, LSTM2

model = LSTM2(input_size, hidden_size, output_size, num_layers)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="max", patience=2, threshold=1, factor=0.8, threshold_mode="abs", min_lr=0.000005, verbose=True)


/Users/alorthius/anaconda3/envs/big_data/lib/python3.10/site-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [15]:
for epoch in range(num_epochs):
    train_loss = 0
    train_corrects = 0

    model.train()
    for inputs, labels in tqdm(train_loader):
        labels = torch.argmax(labels, dim=1)

        optimizer.zero_grad()

        outputs = model(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        train_corrects += torch.sum(preds == labels).item()

    train_loss /= len(train_loader)
    train_acc = train_corrects / len(train_set) * 100
    print(f"[ Train    {epoch + 1}/{num_epochs} ] loss: {train_loss:.4f}, accuracy: [ {train_acc:<7.4f}% ]")

    test_corrects = 0

    model.eval()
    with torch.no_grad():
        for inputs, labels in test_loader:
            labels = torch.argmax(labels, dim=1)

            outputs = model(inputs)

            _, preds = torch.max(outputs, 1)
            test_corrects += torch.sum(preds == labels).item()

    test_acc = test_corrects / len(test_set) * 100
    print(f"[ Gt         {epoch + 1}/{num_epochs} ], accuracy: [ {test_acc:<7.4f}% ]")

    print("-" * 150)

    scheduler.step(test_acc)

100%|██████████| 346/346 [00:20<00:00, 16.68it/s]


[ Train    1/50 ] loss: 1.1516, accuracy: [ 55.7682% ]
[ Gt         1/50 ], accuracy: [ 50.8794% ]
------------------------------------------------------------------------------------------------------------------------------------------------------


100%|██████████| 346/346 [00:22<00:00, 15.19it/s]


[ Train    2/50 ] loss: 0.8312, accuracy: [ 66.6979% ]
[ Gt         2/50 ], accuracy: [ 50.5439% ]
------------------------------------------------------------------------------------------------------------------------------------------------------


100%|██████████| 346/346 [00:21<00:00, 16.02it/s]


...

100%|██████████| 346/346 [00:20<00:00, 16.55it/s]


[ Train    44/50 ] loss: 0.3202, accuracy: [ 87.2930% ]
[ Gt         44/50 ], accuracy: [ 75.5080% ]
------------------------------------------------------------------------------------------------------------------------------------------------------


100%|██████████| 346/346 [00:21<00:00, 16.25it/s]


[ Train    45/50 ] loss: 0.3192, accuracy: [ 87.3248% ]
[ Gt         45/50 ], accuracy: [ 74.6632% ]
------------------------------------------------------------------------------------------------------------------------------------------------------
Epoch 00045: reducing learning rate of group 0 to 1.0737e-04.


 25%|██▍       | 85/346 [00:05<00:16, 16.21it/s]

KeyboardInterrupt



In [ ]:
torch.save(model.state_dict(), f"../microservices/DataManager/weights/multiclass_more/lstm.pth")